<a href="https://colab.research.google.com/github/cbonnin88/VifStream/blob/main/vifStream_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
from google.cloud import bigquery
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats

In [100]:
auth.authenticate_user()
print('Authenticated Successfully')

Authenticated Successfully


# **Initialize the BigQuery Client**

In [102]:
project_id = 'poetic-maxim-486609-e5'
client = bigquery.Client(project=project_id)

# **Pulling the data into a Pandas DataFrame (first) - BQ Client Standard**

In [103]:
query = """
SELECT * FROM `poetic-maxim-486609-e5.vifstream_dbt.fct_user_retention`
WHERE signup_date >= '2025-01-01'
"""
df_pandas_temp = client.query(query).to_dataframe()
df_fct = pl.from_pandas(df_pandas_temp)

print(f'Successfully pulled {df_fct.height} rows from BigQuery')
display(df_fct.head())

Successfully pulled 548986 rows from BigQuery


event_id,user_id,event_name,event_at,signup_date,plan_type,device,days_since_signup
i64,i64,str,"datetime[μs, UTC]",date,str,str,i64
134920,19389,"""add_to_favorites""",2025-12-04 14:07:04.038319 UTC,2025-01-25,"""free""","""Mobile""",313
592737,41747,"""add_to_favorites""",2025-09-19 15:04:54.974391 UTC,2025-04-02,"""free""","""Mobile""",170
359413,3974,"""add_to_favorites""",2025-12-26 20:16:38.554958 UTC,2025-10-10,"""free""","""Mobile""",77
402682,35146,"""add_to_favorites""",2026-01-05 03:26:35.243159 UTC,2026-01-18,"""premium""","""Mobile""",-13
769334,44524,"""add_to_favorites""",2025-11-01 05:25:11.329801 UTC,2025-10-12,"""free""","""Mobile""",20


# Cohort Analysis (Retention Heatmap)

In [104]:
# 2. Transform Data for Cohort Analysis
# Truncate dates to Month for grouping
df_cohort = df_fct.with_columns([
    pl.col('signup_date').cast(pl.Date).dt.truncate('1mo').alias('cohort_month'),
    pl.col('event_at').cast(pl.Date).dt.truncate('1mo').alias('activity_month')
])

df_cohort = df_cohort.with_columns([
    (((pl.col('activity_month') - pl.col('cohort_month')).dt.total_days() / 30).round(0).cast(pl.Int32)).alias('month_number')
])

# Filter out edge cases and limit to first 6 months for clarity
df_cohort = df_cohort.filter((pl.col("month_number") >= 0) & (pl.col("month_number") <= 6))

display(df_cohort.head())

event_id,user_id,event_name,event_at,signup_date,plan_type,device,days_since_signup,cohort_month,activity_month,month_number
i64,i64,str,"datetime[μs, UTC]",date,str,str,i64,date,date,i32
592737,41747,"""add_to_favorites""",2025-09-19 15:04:54.974391 UTC,2025-04-02,"""free""","""Mobile""",170,2025-04-01,2025-09-01,5
359413,3974,"""add_to_favorites""",2025-12-26 20:16:38.554958 UTC,2025-10-10,"""free""","""Mobile""",77,2025-10-01,2025-12-01,2
402682,35146,"""add_to_favorites""",2026-01-05 03:26:35.243159 UTC,2026-01-18,"""premium""","""Mobile""",-13,2026-01-01,2026-01-01,0
769334,44524,"""add_to_favorites""",2025-11-01 05:25:11.329801 UTC,2025-10-12,"""free""","""Mobile""",20,2025-10-01,2025-11-01,1
216190,7512,"""add_to_favorites""",2025-10-19 13:25:41.885886 UTC,2025-05-04,"""free""","""Mobile""",168,2025-05-01,2025-10-01,5


In [105]:
# 3. Aggregate Unique Users
cohort_counts = df_cohort.group_by(['cohort_month','month_number']).agg(
    pl.col('user_id').n_unique().alias('active_users')
).sort(['cohort_month','month_number'])

display(cohort_counts.head())

cohort_month,month_number,active_users
date,i32,u32
2025-01-01,1,1506
2025-01-01,2,1769
2025-01-01,3,1728
2025-01-01,4,1739
2025-01-01,5,1752


In [106]:
# 4. Calucate Retention Percentage
# Get the base size for each cohort (Month 0)
cohort_sizes = cohort_counts.filter(pl.col('month_number')==0).select([
    pl.col('cohort_month'),
    pl.col('active_users').alias('cohort_size')
])


display(cohort_sizes.head())

cohort_month,cohort_size
date,u32
2025-02-01,1421
2025-03-01,1695
2025-04-01,1638
2025-05-01,1756
2025-06-01,1600


In [107]:
# Join back and compute percentage
cohort_final = cohort_counts.join(cohort_sizes, on='cohort_month')
cohort_final = cohort_final.with_columns([
    (pl.col('active_users') / pl.col('cohort_size')* 100).round(1).alias('retention_pct')
])

display(cohort_final.head())

cohort_month,month_number,active_users,cohort_size,retention_pct
date,i32,u32,u32,f64
2025-02-01,0,1421,1421,100.0
2025-02-01,1,1592,1421,112.0
2025-02-01,2,1583,1421,111.4
2025-02-01,3,1641,1421,115.5
2025-02-01,4,1612,1421,113.4


In [108]:
# 5. Pivot for Plotly Heatmap
pivot_pl = cohort_final.pivot(
    index='cohort_month',
    on='month_number',
    values='retention_pct'
).sort('cohort_month')

display(pivot_pl.head())

cohort_month,0,1,2,3,4,5,6
date,f64,f64,f64,f64,f64,f64,f64
2025-02-01,100.0,112.0,111.4,115.5,113.4,111.8,114.2
2025-03-01,100.0,99.5,101.5,98.6,99.8,99.4,99.8
2025-04-01,100.0,103.4,98.0,101.2,102.1,97.8,100.4
2025-05-01,100.0,98.3,97.7,99.1,100.2,100.2,98.6
2025-06-01,100.0,101.2,101.2,100.0,101.5,101.2,101.2


In [109]:
# Convert to Pandas fro Plotly Compatibility
plot_df = pivot_pl.to_pandas().set_index('cohort_month')

In [110]:
fig_cohort = go.Figure(data=go.Heatmap(
    z=plot_df.values,
    x=[f'Month {int(c)}' for c in plot_df.columns],
    y=plot_df.index.astype(str),
    colorscale='Blues',
    text=plot_df.values,
    texttemplate='%{text}%',
    hoverinfo='z'
))

fig_cohort.update_layout(
    title='VifStream: Monthly User Retention (%)',
    xaxis_title='Months After Signup',
    yaxis_title='Signup Month',
    template='plotly_white'
)

fig_cohort.show()

# **Data Visualization with Plotly**

In [22]:
# Definin the stages of the funnel
stages = ['app_open','search','paywall_view','channel_tune_in']

In [23]:
funnel_counts = []
for stage in stages:
  count = df_fct.filter(pl.col('event_name')== stage).select(pl.col('user_id').n_unique()).item()
  funnel_counts.append(count)

In [24]:
fig_funnel = px.funnel(
    data_frame={'Stage':stages,'Users':funnel_counts},
    y='Stage',
    x='Users',
    title='VifStream Onboarding Conversion Funnel',
    color_discrete_sequence=px.colors.qualitative.Prism
)

fig_funnel.show()

# **Segmenting 'Tune-In' Behavior**

In [26]:
# Aggregating events by device and type
device_engagement = df_fct.group_by(['device','event_name']).agg(
    pl.count('event_id').alias('event_count')
).sort('event_count',descending=True)

display(device_engagement.head())

device,event_name,event_count
str,str,u32
"""Mobile""","""channel_tune_in""",187826
"""AppleTV""","""channel_tune_in""",76650
"""Mobile""","""app_open""",75338
"""SmartTV""","""channel_tune_in""",74329
"""Mobile""","""search""",56020


In [30]:
fig_engagement = px.bar(
    device_engagement.to_pandas(),
    x='device',
    y='event_count',
    color='event_name',
    barmode='group',
    title='VifStream Engagement: Event Distribution by Device',
    template='plotly_white',
    labels={'event_count':'Event','device':'Device Type'}
)

fig_engagement.show()

# **Average Time to 'Aha' Moment**

In [34]:
# Filter for first tune-in only
first_tune_in = df_fct.filter(pl.col('event_name') == 'channel_tune_in').group_by('user_id').agg(pl.col('days_since_signup').min().alias('first_tune_day'), pl.col('plan_type').first())

display(first_tune_in)

user_id,first_tune_day,plan_type
i64,i64,str
35014,225,"""free"""
18451,72,"""free"""
42528,375,"""basic"""
20684,54,"""free"""
4356,20,"""free"""
…,…,…
30168,53,"""basic"""
40788,70,"""basic"""
43117,46,"""free"""


In [38]:
# Calculate average per plan
aha_moment_stats = first_tune_in.group_by('plan_type').agg(
    pl.col('first_tune_day').mean().round(0).alias('avg_days_to_tune_in')
)
display(aha_moment_stats)

plan_type,avg_days_to_tune_in
str,f64
"""free""",132.0
"""premium""",131.0
"""basic""",129.0


In [47]:
fig_aha = px.bar(
    aha_moment_stats.to_pandas(),
    x='plan_type',
    y='avg_days_to_tune_in',
    color='plan_type',
    title='Average Days to First Content Engagment by Plan',
    labels={'plan_type':'Subscription Type','avg_days_to_tune_in':'Average Days to Tune-In'},
    text_auto='.0f',
    template='plotly_dark'
)

fig_aha.show()

# **Distribution**

In [44]:
aha_distribution = df_fct.filter(pl.col('event_name')== 'channel_tune_in').group_by('user_id').agg([
    pl.col('days_since_signup').min().alias('days_to_aha'),
    pl.col('plan_type').first()
])

display(aha_distribution.head())

user_id,days_to_aha,plan_type
i64,i64,str
13399,281,"""free"""
4389,33,"""free"""
4261,0,"""free"""
35055,269,"""basic"""
13860,318,"""basic"""


In [45]:
fig_hist = px.histogram(
    aha_distribution.to_pandas(),
    x='days_to_aha',
    color='plan_type',
    nbins=30,
    title='Distribution of Days to Aha! Moments (First Tune-In)',
    labels={'days_to_aha':'Days Since Signup','count':'Number of Users'},
    barmode='overlay',
    template='plotly_white'
)

fig_hist.add_vline(x=aha_distribution['days_to_aha'].mean(),line_dash='dash',line_color='red',annotation_text='Average')

fig_hist.show()